In [8]:
#%pip install msal

StatementMeta(, , -1, Finished, Available)


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [9]:
import msal
import json
from datetime import datetime, timedelta
import os
import requests
from pathlib import Path
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
from delta.tables import *

StatementMeta(, dea877be-e14e-4d3c-8589-8cfd00287b95, 21, Finished, Available)

In [10]:
# The base function is as follows: mssparkutils.credentials.getSecret('azure key vault name','secret name') 

# Please change the below values to your Key Vault URI and Secret Names (if different) 

client_id = mssparkutils.credentials.getSecret('https://algdev-scannerapi.vault.azure.net/','azure-client-id')
client_secret = mssparkutils.credentials.getSecret('https://algdev-scannerapi.vault.azure.net/','azure-client-secret') 
tenant_id = mssparkutils.credentials.getSecret('https://algdev-scannerapi.vault.azure.net/','azure-tenant-id') 


StatementMeta(, dea877be-e14e-4d3c-8589-8cfd00287b95, 22, Finished, Available)

In [11]:
# Define a function to authenticate using service principal. The result contains the token that is used to authenticate via API URL
def power_bi_authenticate_service_principal(tenant_id, client_id, client_secret):
    auth_url = f"https://login.microsoftonline.com/{tenant_id}"
    scope = ["https://analysis.windows.net/powerbi/api/.default"]


    app = msal.ConfidentialClientApplication(client_id = client_id, authority=auth_url, client_credential=client_secret)
    result = app.acquire_token_for_client(scopes=scope)

    # implement future check and troubleshooting if token not found
    # if 'access_token' in result:
    #     access_token = result['access_token']

    return result

StatementMeta(, dea877be-e14e-4d3c-8589-8cfd00287b95, 23, Finished, Available)

In [12]:
# Define a function to make Power BI REST API requests using token
def invoke_power_bi_rest_method(url, token):
    headers = {'Content-Type':'application/json', 'Authorization':f'Bearer {token}'}
    response = requests.get(url, headers=headers)
    return response.json()

StatementMeta(, dea877be-e14e-4d3c-8589-8cfd00287b95, 24, Finished, Available)

In [13]:
# Main function
def main():
    try:
        print("Starting Power BI Activity Fetch")

        stopwatch = datetime.now()

        # Set lookback timeframe to 30 days
        pivot_date = datetime.utcnow().date() - timedelta(days=30)

        # Call function to request token to authenticate via API URL
        pbi_account = power_bi_authenticate_service_principal(tenant_id, client_id, client_secret)

        # Gets API activityevents data for each day in the last 30 days
        while pivot_date <= datetime.utcnow().date():
            print(f"Getting activityevents data for: '{pivot_date.strftime('%Y%m%d')}'")

            # API URL requires dates in ISO8601 format
            activity_api_url = f"https://api.powerbi.com/v1.0/myorg/admin/activityevents?startDateTime='{pivot_date}T00:00:00Z'&endDateTime='{(pivot_date)}T23:59:59Z'"

            activityevents = []

            # Iterate over activity_api_url using continuationUri, since API call has limited output per call (pagination limitation)
            while True:
                result = invoke_power_bi_rest_method(activity_api_url, pbi_account['access_token'])

                if result['activityEventEntities'] is not None:
                    activityevents.extend(result['activityEventEntities'])

                if result['continuationUri'] is not None:
                    activity_api_url = result['continuationUri']
                else:
                    break

            pivot_date += timedelta(days=1)

            if len(activityevents) != 0:
                # Due to Schema issues, we set datatype of all columns to string and format CreationTime to datetime in ISO8601
                activityevents = pd.DataFrame(activityevents).astype(str)
                activityevents['CreationTime'] = pd.to_datetime(activityevents['CreationTime'], format='ISO8601')

                #Check if table already exists in Lakehouse and merge, else write (overwrite) table with current data
                if DeltaTable.isDeltaTable(spark, 'Tables/activityevents'):
                    activityevents = spark.createDataFrame(activityevents)
                    current_activityevents = DeltaTable.forName(spark, "activityevents")
                    spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled","true")
                    current_activityevents.alias("current").merge(activityevents.alias("new"), "current.Id = new.Id AND current.CreationTime = new.CreationTime") \
                    .whenMatchedUpdateAll() \
                    .whenNotMatchedInsertAll() \
                    .execute()

                else:
                    spark.createDataFrame(activityevents).write.mode("overwrite").option("mergeSchema", "true").format("delta").save("Tables/activityevents")
            else:
                print(f"No activityevents logs for date: '{(pivot_date - timedelta(days=1)).strftime('%Y%m%d')}'")
        
    finally:
        elapsed_time = (datetime.now() - stopwatch).total_seconds()
        print(f"Elapsed: {elapsed_time}s")



StatementMeta(, dea877be-e14e-4d3c-8589-8cfd00287b95, 25, Finished, Available)

In [14]:
main()

StatementMeta(, dea877be-e14e-4d3c-8589-8cfd00287b95, 26, Finished, Available)

Starting Power BI Activity Fetch
Getting activityevents data for: '20240130'
No activityevents logs for date: '20240130'
Getting activityevents data for: '20240131'
Getting activityevents data for: '20240201'
Getting activityevents data for: '20240202'
Getting activityevents data for: '20240203'
Getting activityevents data for: '20240204'
Getting activityevents data for: '20240205'
Getting activityevents data for: '20240206'
Getting activityevents data for: '20240207'
Getting activityevents data for: '20240208'
Getting activityevents data for: '20240209'
Getting activityevents data for: '20240210'
Getting activityevents data for: '20240211'
Getting activityevents data for: '20240212'
Getting activityevents data for: '20240213'
Getting activityevents data for: '20240214'
Getting activityevents data for: '20240215'
Getting activityevents data for: '20240216'
Getting activityevents data for: '20240217'
Getting activityevents data for: '20240218'
Getting activityevents data for: '20240219'